In [81]:
import numpy as np
import heapq
from collections import namedtuple
from random import choice
from icecream import ic
import itertools
import time

In [82]:
PUZZLE_DIM = 7
Action = namedtuple('Action', ['pos1', 'pos2'])

In [83]:
def available_actions(state: np.ndarray) -> list[Action]:
    # Find the position of zero (empty space)
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    
    # Optimisation: use a list of directions instead of separate ifs
    directions = [
        (-1, 0),  # su
        (1, 0),   # giù
        (0, -1),  # sinistra
        (0, 1)    # destra
    ]
    
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < PUZZLE_DIM and 0 <= new_y < PUZZLE_DIM:
            actions.append(Action((x, y), (new_x, new_y)))
            
    return actions

def do_action(state: np.ndarray, act: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[act.pos1], new_state[act.pos2] = new_state[act.pos2], new_state[act.pos1]
    return new_state

In [84]:
def heuristic(state: np.ndarray)-> int:
    # Manhattan distance for each puzzle piece
    distance=0
    goal_state=np.array([i for i in range(1,PUZZLE_DIM**2)]+[0]).reshape((PUZZLE_DIM,PUZZLE_DIM)) #values in ascending order, with 0 in the last position
    target_positions = {
        value: divmod(value-1, PUZZLE_DIM) 
        for value in range(1, PUZZLE_DIM**2)
    }
    
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # ignore empty space
                target_x, target_y = target_positions[value]
                distance += abs(x - target_x) + abs(y - target_y)
                
    return distance

In [85]:
def a_star(initial_state: np.ndarray) -> list[Action]:
    open_set = []
    counter = itertools.count()  # Counter for differentiating states in the heap
    heapq.heappush(open_set, (0, next(counter), initial_state, []))  # (priorità, conteggio, stato, percorso)
    visited = set()
    num_tot_action=0
    while open_set:
        cost, _, state, path = heapq.heappop(open_set)
        num_tot_action+=1
        # Check if we have reached the target status
        if np.array_equal(state, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))):
            return path,state,num_tot_action
        
        visited.add(state.tobytes())
        
        # Generate neighbouring states
        for act in available_actions(state):
            new_state = do_action(state, act)
            if new_state.tobytes() not in visited:
                new_cost = len(path) + 1 + heuristic(new_state)
                heapq.heappush(open_set, (new_cost, next(counter), new_state, path + [act]))
    
    return None

In [86]:
def generate_puzzle(num_moves=50):
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    
    for _ in range(num_moves):
        acts = available_actions(state)
        state = do_action(state, choice(acts))
    
    return state

In [87]:
state = generate_puzzle(100)
ic(state)
None

ic| state: array([[ 1,  2,  3,  4,  5,  6,  7],
                  [ 8,  9, 10, 11, 12, 13, 14],
                  [15, 16, 17, 18, 19, 20, 21],
                  [22, 23, 24, 25, 26, 27, 28],
                  [30, 31, 32, 36, 40, 47, 42],
                  [29, 44, 39, 37, 41, 33, 34],
                  [43, 45,  0, 38, 46, 48, 35]])


In [88]:
start_time = time.time()
solution = a_star(state)
end_time  = time.time()

if solution:
    print(f"\nSolution found in {end_time - start_time:.2f} seconds!")
    print("Solution found:", solution[0])
    print("Number of moves(Quality):", len(solution[0]))
    print("Solution:",solution[1])
    print("Cost:", solution[2])
else:
    print("Solution not found.")


Solution found in 12.63 seconds!
Solution found: [Action(pos1=(6, 2), pos2=(5, 2)), Action(pos1=(5, 2), pos2=(5, 3)), Action(pos1=(5, 3), pos2=(4, 3)), Action(pos1=(4, 3), pos2=(4, 4)), Action(pos1=(4, 4), pos2=(4, 5)), Action(pos1=(4, 5), pos2=(5, 5)), Action(pos1=(5, 5), pos2=(5, 4)), Action(pos1=(5, 4), pos2=(4, 4)), Action(pos1=(4, 4), pos2=(4, 3)), Action(pos1=(4, 3), pos2=(4, 2)), Action(pos1=(4, 2), pos2=(5, 2)), Action(pos1=(5, 2), pos2=(5, 3)), Action(pos1=(5, 3), pos2=(6, 3)), Action(pos1=(6, 3), pos2=(6, 2)), Action(pos1=(6, 2), pos2=(6, 1)), Action(pos1=(6, 1), pos2=(5, 1)), Action(pos1=(5, 1), pos2=(5, 2)), Action(pos1=(5, 2), pos2=(4, 2)), Action(pos1=(4, 2), pos2=(4, 1)), Action(pos1=(4, 1), pos2=(4, 0)), Action(pos1=(4, 0), pos2=(5, 0)), Action(pos1=(5, 0), pos2=(5, 1)), Action(pos1=(5, 1), pos2=(5, 2)), Action(pos1=(5, 2), pos2=(5, 3)), Action(pos1=(5, 3), pos2=(6, 3)), Action(pos1=(6, 3), pos2=(6, 4)), Action(pos1=(6, 4), pos2=(5, 4)), Action(pos1=(5, 4), pos2=(4, 4)